# SASRec with negative sampling

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict

# Параметры
max_len = 50        # Максимальная длина последовательности
batch_size = 128    # Размер батча
num_negatives = 300  # Количество негативных сэмплов

# Файлы данных
train_file = '../data/source/1_ml-1m_original.part1.inter'
valid_file = '../data/source/1_ml-1m_original.part2.inter'
test_file = '../data/source/1_ml-1m_original.part3.inter'

# Загрузка данных
train_data = pd.read_csv(train_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)
valid_data = pd.read_csv(valid_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)
test_data = pd.read_csv(test_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)

print(f'Train size: {train_data.shape}')
print(f'Valid size: {valid_data.shape}')
print(f'Test size: {test_data.shape}')

# Для отображения первых строк DataFrame (только если вы работаете в Jupyter Notebook)
# display(train_data.head())

# Подготовка последовательностей для обучения, валидации и теста
def prepare_sequences(data):
    user_group = data.groupby('user_id')['item_id'].apply(list)
    sequences = []
    user_ids = []
    for user_id, seq in user_group.items():
        if len(seq) >= 2:  # Только пользователи с достаточной историей
            sequences.append(seq)
            user_ids.append(user_id)
    return sequences, user_ids

train_sequences, train_user_ids = prepare_sequences(train_data)
valid_sequences, valid_user_ids = prepare_sequences(valid_data)
test_sequences, test_user_ids = prepare_sequences(test_data)

print(f'Количество пользователей в обучающем наборе: {len(train_sequences)}')
print(f'Количество пользователей в валидационном наборе: {len(valid_sequences)}')
print(f'Количество пользователей в тестовом наборе: {len(test_sequences)}')

# Создание набора всех элементов
num_items = max(train_data['item_id'].max(), valid_data['item_id'].max(), test_data['item_id'].max())
all_items = set(range(1, num_items + 1))

# Создание словаря взаимодействий пользователей из всех наборов данных
user_interactions = defaultdict(set)

# Добавляем взаимодействия из обучающего набора
for user_id, group in train_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Добавляем взаимодействия из валидационного набора
for user_id, group in valid_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Добавляем взаимодействия из тестового набора
for user_id, group in test_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Функция для выборки негативных сэмплов
def sample_negatives(user_id, num_negatives, all_items, user_interactions):
    user_items = user_interactions.get(user_id, set())
    negatives = []
    while len(negatives) < num_negatives:
        neg_item = np.random.randint(1, num_items + 1)
        if neg_item not in user_items:
            negatives.append(neg_item)
    return negatives

class MovieLensDataset(Dataset):
    def __init__(self, sequences, max_len=50):
        self.sequences = sequences
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]

        # Паддинг последовательности
        if len(seq) < self.max_len:
            padded_seq = [0] * (self.max_len - len(seq)) + seq
            seq_len = len(seq)
        else:
            padded_seq = seq[-self.max_len:]
            seq_len = self.max_len

        return torch.tensor(padded_seq, dtype=torch.long), torch.tensor(seq_len, dtype=torch.long)

# Создание датасетов и DataLoader
train_dataset = MovieLensDataset(train_sequences, max_len)
valid_dataset = MovieLensDataset(valid_sequences, max_len)
test_dataset = MovieLensDataset(test_sequences, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Класс EvaluationDataset для валидации и тестирования с негативными сэмплами
class EvaluationDataset(Dataset):
    def __init__(self, sequences, user_ids, user_interactions, all_items, num_negatives=1000, max_len=50):
        self.sequences = sequences
        self.user_ids = user_ids
        self.user_interactions = user_interactions
        self.all_items = all_items
        self.num_negatives = num_negatives
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        user_id = self.user_ids[idx]
        pos_item = seq[-1]
        seq_input = seq[:-1]

        # Паддинг последовательности
        if len(seq_input) < self.max_len:
            padded_seq = [0] * (self.max_len - len(seq_input)) + seq_input
            seq_len = len(seq_input)
        else:
            padded_seq = seq_input[-self.max_len:]
            seq_len = self.max_len

        # Выборка негативных сэмплов
        negatives = sample_negatives(user_id, self.num_negatives, self.all_items, self.user_interactions)

        # Список для оценки: 1 положительный + num_negatives негативных элементов
        items = negatives + [pos_item]

        return (torch.tensor(padded_seq, dtype=torch.long),
                torch.tensor(seq_len, dtype=torch.long),
                torch.tensor(items, dtype=torch.long),
                torch.tensor(pos_item, dtype=torch.long))

# Создание EvaluationDataset для валидации и тестирования
valid_eval_dataset = EvaluationDataset(valid_sequences, valid_user_ids, user_interactions, all_items, num_negatives=num_negatives, max_len=max_len)
test_eval_dataset = EvaluationDataset(test_sequences, test_user_ids, user_interactions, all_items, num_negatives=num_negatives, max_len=max_len)

# Используем batch_size=1 для удобства обработки по пользователям
valid_eval_loader = DataLoader(valid_eval_dataset, batch_size=1, shuffle=False)
test_eval_loader = DataLoader(test_eval_dataset, batch_size=1, shuffle=False)

class SASRec(nn.Module):
    def __init__(self, num_items, embedding_dim=50, num_heads=2, num_layers=2, dropout=0.2, max_len=50):
        super(SASRec, self).__init__()
        self.item_embedding = nn.Embedding(num_items + 1, embedding_dim, padding_idx=0)  # +1 для паддинга
        self.position_embedding = nn.Embedding(max_len, embedding_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim,
                                                   nhead=num_heads,
                                                   dropout=dropout,
                                                   activation='relu')
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(embedding_dim, num_items + 1)

    def forward(self, input_seq, seq_len):
        # input_seq: (batch_size, max_len)
        position_ids = torch.arange(0, input_seq.size(1), device=input_seq.device).unsqueeze(0).expand_as(input_seq)
        item_emb = self.item_embedding(input_seq) + self.position_embedding(position_ids)

        item_emb = self.layer_norm(item_emb)
        item_emb = self.dropout(item_emb)

        # Transformer ожидает ввод формы (seq_len, batch_size, embedding_dim)
        item_emb = item_emb.transpose(0, 1)

        # Создание маски для паддинга
        mask = (input_seq == 0)  # (batch_size, max_len)

        # Передача через Transformer
        output = self.transformer(item_emb, src_key_padding_mask=mask)
        output = output.transpose(0, 1)  # (batch_size, max_len, embedding_dim)

        # Предсказание последнего элемента
        output = output[:, -1, :]  # (batch_size, embedding_dim)
        logits = self.fc(output)    # (batch_size, num_items + 1)
        return logits

# Параметры модели
embedding_dim = 50
num_heads = 2
num_layers = 2
dropout = 0.2

model = SASRec(num_items=num_items, embedding_dim=embedding_dim, num_heads=num_heads,
              num_layers=num_layers, dropout=dropout, max_len=max_len)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Критерий и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Игнорируем паддинг
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Функции метрик
def precision_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    hits = len(set(recommended) & set(relevant))
    return hits / k

def recall_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    hits = len(set(recommended) & set(relevant))
    return hits / len(relevant) if relevant else 0

def ndcg_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    dcg = 0.0
    for i, item in enumerate(recommended):
        if item in relevant:
            dcg += 1 / np.log2(i + 2)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0

# Функция обучения
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in loader:
        sequences, lengths = batch
        sequences = sequences.to(device)
        lengths = lengths.to(device)

        optimizer.zero_grad()
        outputs = model(sequences, lengths)
        targets = sequences[:, -1]  # Последний элемент последовательности
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

# Функция валидации
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            sequences, lengths = batch
            sequences = sequences.to(device)
            lengths = lengths.to(device)

            outputs = model(sequences, lengths)
            targets = sequences[:, -1]
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(loader)

# Функция оценки модели с негативными сэмплами
def evaluate_model_with_negatives(model, loader, device, k=10):
    model.eval()
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    with torch.no_grad():
        for batch in loader:
            # Каждый батч содержит одного пользователя
            seq, seq_len, items, pos_item = batch
            seq = seq.to(device)
            seq_len = seq_len.to(device)
            items = items.to(device)  # (batch_size=1, 1001)
            pos_item = pos_item.to(device)  # (batch_size=1)

            # Получение эмбеддингов и прогнозов модели для всех элементов
            outputs = model(seq, seq_len)  # (batch_size=1, num_items +1)

            # Извлечение оценок только для выбранных элементов (негативные + положительный)
            # Предполагается, что items содержат индексы элементов
            item_scores = outputs.gather(1, items)  # (batch_size=1, 1001)

            # Получение top-K элементов среди негативных и положительного
            _, topk_indices = torch.topk(item_scores, k, dim=1)  # (batch_size=1, k)
            topk_items = items[0][topk_indices[0]].cpu().numpy()

            # Положительный элемент находится в конце списка items
            recommended = topk_items
            relevant = [pos_item.item()]

            precision_scores.append(precision_at_k(recommended, relevant, k))
            recall_scores.append(recall_at_k(recommended, relevant, k))
            ndcg_scores.append(ndcg_at_k(recommended, relevant, k))

    # Вычисление средних значений метрик
    mean_precision = np.mean(precision_scores)
    mean_recall = np.mean(recall_scores)
    mean_ndcg = np.mean(ndcg_scores)

    return mean_precision, mean_recall, mean_ndcg

# Цикл обучения с валидацией
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    valid_loss = validate(model, valid_loader, criterion, device)
    print(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')

# Параметр K для top-K рекомендаций
k = 10

# Оценка модели на тестовом наборе с негативными сэмплами
precision, recall, ndcg = evaluate_model_with_negatives(model, test_eval_loader, device, k=k)
print(f'Precision@{k}: {precision:.4f}')
print(f'Recall@{k}: {recall:.4f}')
print(f'NDCG@{k}: {ndcg:.4f}')


Train size: (697378, 4)
Valid size: (99582, 4)
Test size: (203165, 4)
Количество пользователей в обучающем наборе: 6040
Количество пользователей в валидационном наборе: 5954
Количество пользователей в тестовом наборе: 6040


c:\Users\redpo\Desktop\ITMO\ITMO RESEARCH\sasrec-bert4rec-recsys23\.venv\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Train Loss: 7.9700, Valid Loss: 7.3887
Epoch 2/10, Train Loss: 6.9164, Valid Loss: 6.6426
Epoch 3/10, Train Loss: 6.1473, Valid Loss: 5.8828
Epoch 4/10, Train Loss: 5.3706, Valid Loss: 5.1023
Epoch 5/10, Train Loss: 4.5954, Valid Loss: 4.4545
Epoch 6/10, Train Loss: 3.9272, Valid Loss: 3.8361
Epoch 7/10, Train Loss: 3.3532, Valid Loss: 3.3553
Epoch 8/10, Train Loss: 2.8526, Valid Loss: 2.9449
Epoch 9/10, Train Loss: 2.4329, Valid Loss: 2.6268
Epoch 10/10, Train Loss: 2.0553, Valid Loss: 2.4178
Precision@10: 0.0120
Recall@10: 0.1199
NDCG@10: 0.0571


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict

# Параметры
max_len = 50           # Максимальная длина последовательности
batch_size = 128       # Размер батча
num_negatives = 300   # Количество негативных сэмплов
external_embedding_dim = 1536  # Размерность внешних эмбеддингов
projected_embedding_dim = 300   # Размерность после проекции внешних эмбеддингов
embedding_dim = 50     # Размерность внутренних эмбеддингов модели SASRec
lambda_align = 0.1     # Вес для alignment loss

# Файлы данных
train_file = '../data/source/1_ml-1m_original.part1.inter'
valid_file = '../data/source/1_ml-1m_original.part2.inter'
test_file = '../data/source/1_ml-1m_original.part3.inter'

# Загрузка данных
train_data = pd.read_csv(train_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)
valid_data = pd.read_csv(valid_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)
test_data = pd.read_csv(test_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], skiprows=1)

print(f'Train size: {train_data.shape}')
print(f'Valid size: {valid_data.shape}')
print(f'Test size: {test_data.shape}')

# Подготовка последовательностей для обучения, валидации и теста
def prepare_sequences(data):
    user_group = data.groupby('user_id')['item_id'].apply(list)
    sequences = []
    user_ids = []
    for user_id, seq in user_group.items():
        if len(seq) >= 2:  # Только пользователи с достаточной историей
            sequences.append(seq)
            user_ids.append(user_id)
    return sequences, user_ids

train_sequences, train_user_ids = prepare_sequences(train_data)
valid_sequences, valid_user_ids = prepare_sequences(valid_data)
test_sequences, test_user_ids = prepare_sequences(test_data)

print(f'Количество пользователей в обучающем наборе: {len(train_sequences)}')
print(f'Количество пользователей в валидационном наборе: {len(valid_sequences)}')
print(f'Количество пользователей в тестовом наборе: {len(test_sequences)}')

# Определение количества пользователей и элементов
num_users = max(train_data['user_id'].max(), valid_data['user_id'].max(), test_data['user_id'].max())
num_items = max(train_data['item_id'].max(), valid_data['item_id'].max(), test_data['item_id'].max())

# Создание набора всех элементов
all_items = set(range(1, num_items + 1))

# Создание словаря взаимодействий пользователей из всех наборов данных
user_interactions = defaultdict(set)

# Добавляем взаимодействия из обучающего набора
for user_id, group in train_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Добавляем взаимодействия из валидационного набора
for user_id, group in valid_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Добавляем взаимодействия из тестового набора
for user_id, group in test_data.groupby('user_id'):
    user_interactions[user_id].update(group['item_id'].tolist())

# Функция для выборки негативных сэмплов
def sample_negatives(user_id, num_negatives, all_items, user_interactions):
    user_items = user_interactions.get(user_id, set())
    negatives = []
    while len(negatives) < num_negatives:
        neg_item = np.random.randint(1, num_items + 1)
        if neg_item not in user_items:
            negatives.append(neg_item)
    return negatives

# Загрузка внешних эмбеддингов пользователей
with open('../data/emb/embeddings.json', 'r') as f:
    user_embeddings = json.load(f)

# Преобразование эмбеддингов пользователей в словарь для быстрого доступа
user2embedding = {int(user['id']): user['embedding'] for user in user_embeddings}

# Проверка размерности внешних эмбеддингов
if len(next(iter(user2embedding.values()))) != external_embedding_dim:
    raise ValueError(f"Размерность внешних эмбеддингов ({len(next(iter(user2embedding.values())))}), не совпадает с ожидаемой ({external_embedding_dim})")

# Класс Dataset для обучения и валидации, включающий user_id
class MovieLensDataset(Dataset):
    def __init__(self, sequences, user_ids, max_len=50):
        self.sequences = sequences
        self.user_ids = user_ids
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        user_id = self.user_ids[idx]

        # Паддинг последовательности
        if len(seq) < self.max_len:
            padded_seq = [0] * (self.max_len - len(seq)) + seq
            seq_len = len(seq)
        else:
            padded_seq = seq[-self.max_len:]
            seq_len = self.max_len

        return torch.tensor(user_id, dtype=torch.long), torch.tensor(padded_seq, dtype=torch.long), torch.tensor(seq_len, dtype=torch.long)

# Класс EvaluationDataset для валидации и тестирования с негативными сэмплами
class EvaluationDataset(Dataset):
    def __init__(self, sequences, user_ids, user_interactions, all_items, user2embedding, num_negatives=1000, max_len=50):
        self.sequences = sequences
        self.user_ids = user_ids
        self.user_interactions = user_interactions
        self.all_items = all_items
        self.num_negatives = num_negatives
        self.max_len = max_len
        self.user2embedding = user2embedding

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        user_id = self.user_ids[idx]
        pos_item = seq[-1]
        seq_input = seq[:-1]

        # Паддинг последовательности
        if len(seq_input) < self.max_len:
            padded_seq = [0] * (self.max_len - len(seq_input)) + seq_input
            seq_len = len(seq_input)
        else:
            padded_seq = seq_input[-self.max_len:]
            seq_len = self.max_len

        # Выборка негативных сэмплов
        negatives = sample_negatives(user_id, self.num_negatives, self.all_items, self.user_interactions)

        # Список для оценки: num_negatives негативных элементов + 1 положительный элемент
        items = negatives + [pos_item]

        return (torch.tensor(user_id, dtype=torch.long),
                torch.tensor(padded_seq, dtype=torch.long),
                torch.tensor(seq_len, dtype=torch.long),
                torch.tensor(items, dtype=torch.long),
                torch.tensor(pos_item, dtype=torch.long))

# Создание датасетов и DataLoader для обучения и валидации
train_dataset = MovieLensDataset(train_sequences, train_user_ids, max_len)
valid_dataset = MovieLensDataset(valid_sequences, valid_user_ids, max_len)
test_dataset = MovieLensDataset(test_sequences, test_user_ids, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Создание EvaluationDataset для валидации и тестирования
valid_eval_dataset = EvaluationDataset(valid_sequences, valid_user_ids, user_interactions, all_items, user2embedding, num_negatives=num_negatives, max_len=max_len)
test_eval_dataset = EvaluationDataset(test_sequences, test_user_ids, user_interactions, all_items, user2embedding, num_negatives=num_negatives, max_len=max_len)

# Используем batch_size=1 для удобства обработки по пользователям
valid_eval_loader = DataLoader(valid_eval_dataset, batch_size=1, shuffle=False)
test_eval_loader = DataLoader(test_eval_dataset, batch_size=1, shuffle=False)

# Определение модели SASRec с интеграцией пользовательских эмбеддингов и проекцией внешних эмбеддингов
class SASRec(nn.Module):
    def __init__(self, num_users, num_items, user_embedding_dim=1536, projection_dim=300, embedding_dim=50, num_heads=2, num_layers=2, dropout=0.2, max_len=50, user2embedding=None):
        super(SASRec, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.max_len = max_len

        # Эмбеддинги пользователей
        self.user_embedding = nn.Embedding(num_users + 1, user_embedding_dim, padding_idx=0)
        if user2embedding is not None:
            self.init_user_embeddings(user2embedding)

        # Проекция внешних эмбеддингов пользователей в пространство модели
        self.external_projection = nn.Linear(user_embedding_dim, projection_dim)
        self.user_projection = nn.Linear(projection_dim, embedding_dim)

        # Эмбеддинги элементов и позиций
        self.item_embedding = nn.Embedding(num_items + 1, embedding_dim, padding_idx=0)  # +1 для паддинга
        self.position_embedding = nn.Embedding(max_len, embedding_dim)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim,
                                                   nhead=num_heads,
                                                   dropout=dropout,
                                                   activation='relu')
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(embedding_dim, num_items + 1)

    def init_user_embeddings(self, user2embedding):
        # Инициализация эмбеддингов пользователей из внешних данных
        user_embeddings_tensor = torch.zeros(self.num_users + 1, external_embedding_dim)
        for user_id, embedding in user2embedding.items():
            if 0 < user_id <= self.num_users:
                user_embeddings_tensor[user_id] = torch.tensor(embedding)
        self.user_embedding.weight.data.copy_(user_embeddings_tensor)
        # Если требуется, можно зафиксировать эмбеддинги пользователей
        # self.user_embedding.weight.requires_grad = False

    def forward(self, user_ids, input_seq, seq_len):
        """
        user_ids: (batch_size,)
        input_seq: (batch_size, max_len)
        seq_len: (batch_size,)
        """
        # Получение эмбеддингов пользователей
        user_emb = self.user_embedding(user_ids)        # (batch_size, user_embedding_dim=1536)
        user_emb = self.external_projection(user_emb)   # (batch_size, projection_dim=300)
        user_emb = self.user_projection(user_emb)       # (batch_size, embedding_dim=50)
        user_emb = user_emb.unsqueeze(1)                # (batch_size, 1, embedding_dim=50)

        # Получение эмбеддингов элементов
        item_emb = self.item_embedding(input_seq)       # (batch_size, max_len, embedding_dim=50)

        # Получение эмбеддингов позиций
        position_ids = torch.arange(0, input_seq.size(1), device=input_seq.device).unsqueeze(0).expand_as(input_seq)
        pos_emb = self.position_embedding(position_ids)  # (batch_size, max_len, embedding_dim=50)

        # Комбинирование эмбеддингов элементов, позиций и пользователей
        combined_emb = item_emb + pos_emb                 # (batch_size, max_len, embedding_dim=50)
        # Добавление пользовательского эмбеддинга к каждому элементу последовательности
        user_emb_expanded = user_emb.expand(-1, input_seq.size(1), -1)  # (batch_size, max_len, embedding_dim=50)
        combined_emb = combined_emb + user_emb_expanded             # (batch_size, max_len, embedding_dim=50)

        # Нормализация и дропаут
        combined_emb = self.layer_norm(combined_emb)
        combined_emb = self.dropout(combined_emb)

        # Transformer ожидает ввод формы (seq_len, batch_size, embedding_dim)
        combined_emb = combined_emb.transpose(0, 1)  # (max_len, batch_size, embedding_dim=50)

        # Создание маски для паддинга
        mask = (input_seq == 0)  # (batch_size, max_len)

        # Передача через Transformer
        output = self.transformer(combined_emb, src_key_padding_mask=mask)  # (max_len, batch_size, embedding_dim=50)
        output = output.transpose(0, 1)                                   # (batch_size, max_len, embedding_dim=50)

        # Предсказание последнего элемента
        output = output[:, -1, :]  # (batch_size, embedding_dim=50)
        logits = self.fc(output)    # (batch_size, num_items + 1)

        return logits, output      # Возвращаем логиты и внутренние эмбеддинги пользователей (embedding_dim=50)

# Инициализация модели
model = SASRec(num_users=num_users,
              num_items=num_items,
              user_embedding_dim=external_embedding_dim,  # 1536
              projection_dim=projected_embedding_dim,    # 300
              embedding_dim=embedding_dim,               # 50
              num_heads=2,
              num_layers=2,
              dropout=0.2,
              max_len=max_len,
              user2embedding=user2embedding)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Определение функций потерь
class BPRLoss(nn.Module):
    def __init__(self):
        super(BPRLoss, self).__init__()

    def forward(self, pos_scores, neg_scores):
        """
        pos_scores: (batch_size, 1) — Предсказания для положительных элементов
        neg_scores: (batch_size, 1) — Предсказания для негативных элементов
        """
        # BPR Loss вычисляется как -log(sigmoid(pos - neg))
        return -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))

class CombinedLoss(nn.Module):
    def __init__(self, bpr_loss, alignment_loss, lambda_align=0.1):
        super(CombinedLoss, self).__init__()
        self.bpr_loss = bpr_loss
        self.alignment_loss = alignment_loss
        self.lambda_align = lambda_align

    def forward(self, pos_scores, neg_scores, internal_user_emb, projected_external_emb):
        loss_bpr = self.bpr_loss(pos_scores, neg_scores)
        loss_align = self.alignment_loss(internal_user_emb, projected_external_emb)
        return loss_bpr + self.lambda_align * loss_align

# Инициализация потерь и оптимизатора
bpr_loss = BPRLoss()
alignment_loss = nn.MSELoss()
combined_criterion = CombinedLoss(bpr_loss, alignment_loss, lambda_align=lambda_align)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Функции метрик
def precision_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    hits = len(set(recommended) & set(relevant))
    return hits / k

def recall_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    hits = len(set(recommended) & set(relevant))
    return hits / len(relevant) if relevant else 0

def ndcg_at_k(recommended, relevant, k):
    recommended = recommended[:k]
    dcg = 0.0
    for i, item in enumerate(recommended):
        if item in relevant:
            dcg += 1 / np.log2(i + 2)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0

# Функция обучения с использованием BPR Loss и alignment loss
def train_epoch(model, loader, optimizer, combined_criterion, device):
    model.train()
    total_loss = 0
    for batch in loader:
        user_ids, sequences, lengths = batch
        user_ids = user_ids.to(device)      # (batch_size,)
        sequences = sequences.to(device)    # (batch_size, max_len)
        lengths = lengths.to(device)        # (batch_size,)

        optimizer.zero_grad()

        # Предсказания для всех элементов
        logits, internal_user_emb = model(user_ids, sequences, lengths)  # logits: (batch_size, num_items +1), internal_user_emb: (batch_size, embedding_dim=50)

        # Положительные элементы — последний элемент в последовательности
        pos_items = sequences[:, -1].unsqueeze(1)  # (batch_size, 1)
        pos_scores = logits.gather(1, pos_items)   # (batch_size, 1)

        # Выборка негативных элементов
        neg_items = torch.randint(1, num_items + 1, pos_items.size(), device=device)
        # Убедимся, что негативные элементы действительно негативные
        for i in range(neg_items.size(0)):
            while neg_items[i].item() in user_interactions[user_ids[i].item()]:
                neg_items[i] = torch.randint(1, num_items + 1, (1,), device=device)

        neg_scores = logits.gather(1, neg_items)   # (batch_size, 1)

        # Получение внешних эмбеддингов пользователей и проекция их
        external_emb = torch.tensor([user2embedding[user_id.item()] for user_id in user_ids], dtype=torch.float32).to(device)  # (batch_size, external_embedding_dim=1536)
        projected_external_emb = model.user_projection(model.external_projection(external_emb))  # (batch_size, embedding_dim=50)

        # Вычисление комбинированного лосса
        loss = combined_criterion(pos_scores, neg_scores, internal_user_emb, projected_external_emb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

# Функция валидации с использованием BPR Loss и alignment loss
def validate(model, loader, combined_criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            user_ids, sequences, lengths = batch
            user_ids = user_ids.to(device)
            sequences = sequences.to(device)
            lengths = lengths.to(device)

            # Предсказания для всех элементов
            logits, internal_user_emb = model(user_ids, sequences, lengths)  # (batch_size, num_items +1), (batch_size, embedding_dim=50)

            # Положительные элементы — последний элемент в последовательности
            pos_items = sequences[:, -1].unsqueeze(1)  # (batch_size, 1)
            pos_scores = logits.gather(1, pos_items)   # (batch_size, 1)

            # Выборка негативных элементов
            neg_items = torch.randint(1, num_items + 1, pos_items.size(), device=device)
            # Убедимся, что негативные элементы действительно негативные
            for i in range(neg_items.size(0)):
                while neg_items[i].item() in user_interactions[user_ids[i].item()]:
                    neg_items[i] = torch.randint(1, num_items + 1, (1,), device=device)

            neg_scores = logits.gather(1, neg_items)   # (batch_size, 1)

            # Получение внешних эмбеддингов пользователей и проекция их
            external_emb = torch.tensor([user2embedding[user_id.item()] for user_id in user_ids], dtype=torch.float32).to(device)  # (batch_size, external_embedding_dim=1536)
            projected_external_emb = model.user_projection(model.external_projection(external_emb))  # (batch_size, embedding_dim=50)

            # Вычисление комбинированного лосса
            loss = combined_criterion(pos_scores, neg_scores, internal_user_emb, projected_external_emb)

            total_loss += loss.item()
    return total_loss / len(loader)

# Функция оценки модели с негативными сэмплами
def evaluate_model_with_negatives(model, loader, device, k=10):
    model.eval()
    precision_scores = []
    recall_scores = []
    ndcg_scores = []

    with torch.no_grad():
        for batch in loader:
            # Каждый батч содержит одного пользователя
            user_id, seq, seq_len, items, pos_item = batch
            user_id = user_id.to(device)      # (1,)
            seq = seq.to(device)              # (1, max_len)
            seq_len = seq_len.to(device)      # (1,)
            items = items.to(device)          # (1, num_negatives +1)
            pos_item = pos_item.to(device)    # (1,)

            # Получение эмбеддингов и прогнозов модели для всех элементов
            logits, _ = model(user_id, seq, seq_len)  # (1, num_items +1)

            # Извлечение оценок только для выбранных элементов (негативные + положительный)
            # Предполагается, что items содержат индексы элементов
            item_scores = logits.gather(1, items)  # (1, num_negatives +1)

            # Получение top-K элементов среди негативных и положительного
            _, topk_indices = torch.topk(item_scores, k, dim=1)  # (1, k)
            topk_items = items[0][topk_indices[0]].cpu().numpy()

            # Положительный элемент находится в конце списка items
            recommended = topk_items
            relevant = [pos_item.item()]

            precision_scores.append(precision_at_k(recommended, relevant, k))
            recall_scores.append(recall_at_k(recommended, relevant, k))
            ndcg_scores.append(ndcg_at_k(recommended, relevant, k))

    # Вычисление средних значений метрик
    mean_precision = np.mean(precision_scores)
    mean_recall = np.mean(recall_scores)
    mean_ndcg = np.mean(ndcg_scores)

    return mean_precision, mean_recall, mean_ndcg

# Цикл обучения с валидацией
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, combined_criterion, device)
    valid_loss = validate(model, valid_loader, combined_criterion, device)
    print(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')

# Параметр K для top-K рекомендаций
k = 10

# Оценка модели на тестовом наборе с негативными сэмплами
precision, recall, ndcg = evaluate_model_with_negatives(model, test_eval_loader, device, k=k)
print(f'Precision@{k}: {precision:.4f}')
print(f'Recall@{k}: {recall:.4f}')
print(f'NDCG@{k}: {ndcg:.4f}')


Train size: (697378, 4)
Valid size: (99582, 4)
Test size: (203165, 4)
Количество пользователей в обучающем наборе: 6040
Количество пользователей в валидационном наборе: 5954
Количество пользователей в тестовом наборе: 6040


c:\Users\redpo\Desktop\ITMO\ITMO RESEARCH\sasrec-bert4rec-recsys23\.venv\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Train Loss: 0.7163, Valid Loss: 0.6016
Epoch 2/10, Train Loss: 0.5178, Valid Loss: 0.4936
Epoch 3/10, Train Loss: 0.3916, Valid Loss: 0.4591
Epoch 4/10, Train Loss: 0.3052, Valid Loss: 0.4548
Epoch 5/10, Train Loss: 0.2470, Valid Loss: 0.4477
Epoch 6/10, Train Loss: 0.2078, Valid Loss: 0.4351
Epoch 7/10, Train Loss: 0.1854, Valid Loss: 0.4420
Epoch 8/10, Train Loss: 0.1649, Valid Loss: 0.4383
Epoch 9/10, Train Loss: 0.1558, Valid Loss: 0.4406
Epoch 10/10, Train Loss: 0.1472, Valid Loss: 0.4423
Precision@10: 0.0184
Recall@10: 0.1841
NDCG@10: 0.0917


# 1000 negative sampling

| Метрика         | Baseline | Transfer Learning |
|-----------------|----------|-------------------|
| Precision@10    | 0.0044   | 0.0075           |
| Recall@10       | 0.0444   | 0.0747           |
| NDCG@10         | 0.0211   | 0.0375           |

# 300 negative sampling

| Метрика         | Baseline | Transfer Learning |
|-----------------|----------|-------------------|
| Precision@10    | 0.0120   | 0.0184           |
| Recall@10       | 0.1199   | 0.1841           |
| NDCG@10         | 0.0571   | 0.0917           |
